## Tax calculator

In [2]:
#Employee class
class Employee:
    #constructor of employee class
    def __init__(self, StaffID, LastName, FirstName, RegHours, HourlyRate, OTMultiple, TaxCredit, StandardBand):
        self.StaffID = StaffID
        self.LastName = LastName
        self.FirstName = FirstName
        self.RegHours = RegHours
        self.HourlyRate = HourlyRate
        self.OTMultiple = OTMultiple
        self.TaxCredit = TaxCredit
        self.StandardBand = StandardBand
        
    #Compute payment takes hours worked and date and returns the json
    def computePayment(self, HoursWorked, date):
        o = {}
        o['Hours Worked'] = HoursWorked
        o['Regular Hours Worked'] = self.RegHours
        o['Regular Rate'] = self.HourlyRate
        OTMultiple = self.OTMultiple
        o['Tax Credit'] = self.TaxCredit
        StandardBand = self.StandardBand
        o['Date'] = date
        o['Name'] = self.FirstName + ' ' + self.LastName
        o['Regular Pay'] = o['Regular Rate'] * o['Regular Hours Worked']
        # Regular Hours Worked cannot exceed hours worked
        if (o['Hours Worked'] > o['Regular Hours Worked']):
            o['Overtime Hours Worked'] = o['Hours Worked'] - o['Regular Hours Worked']
        else:
            o['Overtime Hours Worked'] = 0
        o['OverTime Rate'] = int(o['Regular Rate'] * OTMultiple)
        # Overtime pay or overtime hours cannot be negative.
        o['OverTime Pay'] = o['Overtime Hours Worked'] * o['OverTime Rate']
        o['Gross Pay'] = o['Regular Pay'] + o['OverTime Pay']
        o['Standard Rate Pay'] = StandardBand
        if (o['Gross Pay'] > o['Standard Rate Pay']):
            o['Higher Rate Pay'] = o['Gross Pay'] - o['Standard Rate Pay']
        else:
            o['Higher Rate Pay'] = 0
        o['Standard Tax'] = int(o['Standard Rate Pay'] * 0.2)
        # Higher Tax cannot be negative.
        o['Higher Tax'] = round(o['Higher Rate Pay'] * 0.4, 2)
        o['Total Tax'] = round(o['Standard Tax'] + o['Higher Tax'], 2)
        o['Net Deduction'] = round(o['Total Tax'] - o['Tax Credit'], 2)
        # Net Pay cannot be negative.
        o['Net Pay'] = round(o['Gross Pay'] - o['Net Deduction'], 2)
        return o
    
    #static method to perform bulk operation
    @staticmethod
    def bulk_tax_compute(employeeFile, hourFile):
        employdata = {}
        result = []
        try:
            #read employee file
            with open(employeeFile) as newFile:
                for line in newFile:
                    StaffID, LastName, FirstName, RegHours, HourlyRate, OTMultiple, TaxCredit, StandardBand = line.split()
                    employdata[StaffID] = Employee(StaffID, LastName, FirstName, int(RegHours), int(HourlyRate),
                                                   float(OTMultiple),
                                                   int(TaxCredit), int(StandardBand))
            #read hour file
            with open(hourFile) as hoursFile:
                for line in hoursFile:
                    StaffID, date, HoursWorked = line.split()
                    if StaffID in employdata:
                        result.append(employdata[StaffID].computePayment(int(HoursWorked), date))
        except Exception as ex:
            print(ex)
            raise Exception("Invalid Input file")
        return result

### Testing output

In [3]:
#testing output
print("=================Single operation==========================")
employee = Employee(12345, 'Smith', 'Neil', 37, 16, 1.5, 70, 700)
print(employee.computePayment(42, '11/11/21'))

print()
print("=================Bulk operation==========================")

try:
    print(Employee.bulk_tax_compute('Employees.txt','Hours.txt'))
except Exception as ex:
    print(ex)

=================Single operation==========================
{'Hours Worked': 42, 'Regular Hours Worked': 37, 'Regular Rate': 16, 'Tax Credit': 70, 'Date': '11/11/21', 'Name': 'Neil Smith', 'Regular Pay': 592, 'Overtime Hours Worked': 5, 'OverTime Rate': 24, 'OverTime Pay': 120, 'Gross Pay': 712, 'Standard Rate Pay': 700, 'Higher Rate Pay': 12, 'Standard Tax': 140, 'Higher Tax': 4.8, 'Total Tax': 144.8, 'Net Deduction': 74.8, 'Net Pay': 637.2}

=================Bulk operation==========================
[{'Hours Worked': 42, 'Regular Hours Worked': 37, 'Regular Rate': 16, 'Tax Credit': 70, 'Date': '31/10/2021', 'Name': 'Joe Green', 'Regular Pay': 592, 'Overtime Hours Worked': 5, 'OverTime Rate': 24, 'OverTime Pay': 120, 'Gross Pay': 712, 'Standard Rate Pay': 700, 'Higher Rate Pay': 12, 'Standard Tax': 140, 'Higher Tax': 4.8, 'Total Tax': 144.8, 'Net Deduction': 74.8, 'Net Pay': 637.2}, {'Hours Worked': 45, 'Regular Hours Worked': 40, 'Regular Rate': 18, 'Tax Credit': 70, 'Date': '31/10/20

## Unit test case

In [ ]:
import unittest

class TestTaxCalculator(unittest.TestCase):
    
    def test_hours_worked(self):
        employee = Employee(12345, 'Smith', 'Neil', 37, 16, 1.5, 70, 700)
        self.assertEqual(employee.computePayment(42, '11/11/21')['Hours Worked'], 42)
        
    def testNetLessEqualGross(self):
        e=Employee(12345, 'Green', 'Joe', 37, 16, 1.5, 70, 700)
        pi=e.computePayment(1,'31/10/2021')
        self.assertLessEqual(pi['Net Pay'],pi['Gross Pay'])
        
    # Overtime pay or overtime hours cannot be negative.
    def test_overtime_pay_cannot_be_negative(self):
        employee = Employee(12345, 'Smith', 'Neil', 37, 16, 1.5, 70, 700)
        self.assertGreater(employee.computePayment(42, '11/11/21')['OverTime Pay'], 0)
        
        employee1 = Employee(123513, 'Roger', 'Alan', 40, 18, 1.5, 70, 700)
        self.assertGreater(employee1.computePayment(42, '11/11/21')['OverTime Pay'], 0)
        
    # Regular Hours Worked cannot exceed hours worked
    def test_regular_hours_Worked_cannot_exceed_hours_worked(self):
        employee = Employee(12345, 'Smith', 'Neil', 37, 16, 1.5, 70, 700)
        result = employee.computePayment(42, '11/11/21')
        self.assertLess(result['Regular Hours Worked'], result['Hours Worked'])
        
    
    # Higher Tax cannot be negative.
    def test_higher_tax_cannot_be_negative(self):
        employee = Employee(12345, 'Smith', 'Neil', 37, 16, 1.5, 70, 700)
        self.assertGreater(employee.computePayment(42, '11/11/21')['Higher Tax'], 0)
        
        employee1 = Employee(123513, 'Roger', 'Alan', 40, 18, 1.5, 70, 700)
        self.assertGreater(employee1.computePayment(42, '11/11/21')['Higher Tax'], 0)
        
    
    # Net Pay cannot be negative.
    def test_net_pay_cannot_be_negative(self):
        employee = Employee(12345, 'Smith', 'Neil', 37, 16, 1.5, 70, 700)
        self.assertGreater(employee.computePayment(42, '11/11/21')['Net Pay'], 0)
        
        employee1 = Employee(123513, 'Roger', 'Alan', 40, 18, 1.5, 70, 700)
        self.assertGreater(employee1.computePayment(42, '11/11/21')['Net Pay'], 0)
        
unittest.main(argv=[''], verbosity=2, exit=False)